In [9]:
import sys
!{sys.executable} -m pip install --user twitter
#from tweepy.streaming import streamListener
#from tweepy import OAuthHandler
#from tweepy import stream
from twitter import *
#from tkinter import *

ACCESS_TOKEN = "988216913284255744-qBZJ3IAkrjBAPLaMRMjhkQrLtKIzn31"
ACCESS_TOKEN_SECRET = "WpcZY8dTZAQVWEJkZBlS8hMQaR9ckqOs8ErFML2x2bVAk"
CONSUMER_KEY = "sRKZDEri6WsOhDmJmxxbnLvyC"
CONSUMER_SECRET = "OJO5n71PvgNE6B6JLePAlRTyOlDoTFFpWziUEE4stu0pbNNLHo"

def getFollowers(user, degree): #recursive function that access followers
    if degree == 3:
        return 0
    x = t.followers.list(screen_name=user) #dictionary containing list of followers  
    while True:
        for i in range(0,len(x['users'])): #iterate through list of followers
                follower = x['users'][i]['screen_name'] #access follower username
                print(follower)
                if not x['users'][i]['protected']: #recursively get follower's followers if account is not private
                    getFollowers(follower, degree + 1)
        print("ok")
        cursor = x['next_cursor'] #get next page
        if cursor == 0: #no next page
            break
        x = t.followers.list(screen_name=user, cursor = cursor) #turn to next page


# Put in token, token_key, con_secret, con_secret_key
t = Twitter(
    auth=OAuth('988216913284255744-qBZJ3IAkrjBAPLaMRMjhkQrLtKIzn31', 'WpcZY8dTZAQVWEJkZBlS8hMQaR9ckqOs8ErFML2x2bVAk',
               'sRKZDEri6WsOhDmJmxxbnLvyC', 'OJO5n71PvgNE6B6JLePAlRTyOlDoTFFpWziUEE4stu0pbNNLHo'))

getFollowers("dijkstra_edsger", 1)






            



You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
FrancisSchickel
eddieyuan_
night_reider
CrithfieldKelly
AIcoholProbs
ChrisMcDIII
mattkirkle
Biersy_2
jerrybrokke
BrandonCohen98
CourkDork
cris__aleman7
GetsITAndy
johnjdechant
Steve_Tha_GOAT
uknowimWHITE
cdemro1
nroballin
d_zana17
theLifeofJerryB
mommymary1969
Thr33_DDD
ok
Kimberlynne_
NighKayla
MeghaDevaraj
kroll_luke
JeffDeHaven4
tjgerot
Cup_Of_Goffee
Andrew_Fuller_4
edwinruiz42
tonywedral
Fullnamecantfit
laurenhanson_
queer_babe
Pachunka21
hzushwywhsysyg1
nicolejarzobski
Ryan_Lawrence14
kboy4m
JakeRyanScott1
We_LuvFrenchies
ok
joonybjones
ryanphe81264965
kristin_goertz
Aleser1234
durinadila121
RepresentOmaha
BRIng_it_on32
Hanny_Webber
mel_caniglia
mara_bullock
down_bythe_bay
sudafedstevens
LeinenAshley
saltyshailyn
Saebastion
bestdadof2018
bigdjohnsonn1
mshswoodworking
121Brock
lilz7golfer
ok
linnffalk
chad_powpow
KaneSila
WD_Horan
WERTH_it

TwitterHTTPError: Twitter sent status 429 for URL: 1.1/followers/list.json using parameters: (cursor=1478896429778309477&oauth_consumer_key=sRKZDEri6WsOhDmJmxxbnLvyC&oauth_nonce=13494014697333300468&oauth_signature_method=HMAC-SHA1&oauth_timestamp=1524716166&oauth_token=988216913284255744-qBZJ3IAkrjBAPLaMRMjhkQrLtKIzn31&oauth_version=1.0&screen_name=eddieyuan_&oauth_signature=lvOExNDV23OZFzc01apirdDvRus%3D)
details: {'errors': [{'message': 'Rate limit exceeded', 'code': 88}]}